## Import necessary libraries for data cleaning

In [1]:
%pip install pandas numpy

  Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached numpy-2.2.3-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached pytz-2025.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.3-cp312-cp312-win_amd64.whl (11.5 MB)
Using cached numpy-2.2.3-cp312-cp312-win_amd64.whl (12.6 MB)
Using cached pytz-2025.1-py2.py3-none-any.whl (507 kB)
Using cached tzdata-2025.1-py2.py3-none-any.whl (346 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np

## Load dataset and preliminary cleaning

In [46]:
df = pd.read_csv('CityExpensesAndRevenue.csv')
print(df.columns)
rows, columns = df.shape
print(f"Number of rows before cleaning: {rows}")
print(f"Number of columns before cleaning: {columns}")

# Rename columns
df = df.rename(columns={
    '_id': 'id',
    'fund_number': 'fund_id',
    'department_number': 'department_id',
    'ledger_descrpition': 'ledger_description',
    'cost_center_number': 'cost_center_id'
})

# Drop unnecessary columns
df = df.drop(columns=['object_account_number', 'object_account_description'])

# Remove rows that has ledger_description = 'Transfers' or have duplicate foreign key values
df = df[df['ledger_description'] != 'Transfers']
df = df[df['fund_id'] != 78770]
df = df[df['fund_id'] != 40015]
df = df[df['department_id'] != 999]

rows, columns = df.shape
print(f"Number of rows after dropping: {rows}")
print(f"Number of columns after dropping: {columns}")
df.head()

Index(['_id', 'fund_number', 'fund_description', 'department_number',
       'department_name', 'cost_center_number', 'cost_center_description',
       'object_account_number', 'object_account_description',
       'general_ledger_date', 'amount', 'ledger_code', 'ledger_descrpition'],
      dtype='object')
Number of rows before cleaning: 617753
Number of columns before cleaning: 13
Number of rows after dropping: 609499
Number of columns after dropping: 11


,id,fund_id,fund_description,department_id,department_name,cost_center_id,cost_center_description,general_ledger_date,amount,ledger_code,ledger_description
0,77837069,11101,GENERAL FUND,103,Innovation & Performance,103000,INNOVATION & PERFORMANCE,31/12/2020,1543.98,5,Expenses
1,77837070,11101,GENERAL FUND,107,Department of Finance,107000,FINANCE,31/12/2020,4453.96,5,Expenses
2,77837071,11101,GENERAL FUND,107,Department of Finance,107000,FINANCE,31/12/2020,197.56,5,Expenses
3,77837072,11101,GENERAL FUND,230,DPS-Police,230000,PS - POLICE BUREAU,31/12/2020,2243.43,5,Expenses
4,77837073,11101,GENERAL FUND,250,DPS-Fire,250000,PS - FIRE BUREAU,31/12/2020,9655.92,5,Expenses


In [47]:
# Check for null values
null_values = df.isnull().sum()
print("\nNull values in each column:")
print(null_values)

# Check if there are any duplicate values in the 'id' column
duplicates = df['id'].duplicated().sum()

# Print the number of duplicate values (if any)
print(f"Number of duplicate values in 'id' column: {duplicates}")


Null values in each column:
id                         0
fund_id                    0
fund_description           0
department_id              0
department_name            0
cost_center_id             0
cost_center_description    0
general_ledger_date        0
amount                     0
ledger_code                0
ledger_description         0
dtype: int64
Number of duplicate values in 'id' column: 0


In [48]:
# Get unique values for each column
for column in df.columns:
    unique_values = df[column].unique()
    print(f"\nUnique values in column '{column}':")
    print(unique_values)




Unique values in column 'id':
[77837069 77837070 77837071 ... 78454581 78454582 78454583]

Unique values in column 'fund_id':
[11101 28050 28020 29040 24067 28033 28003 26030 28055 28004 23000 78765
 26010 24037 78768 32000 24016 26020 28070 28043 28053 40120 29060 75020
 78160 78170 40017 40018 40119 40019 40118 24063 28085 40001 28045 22000
 40116 40020 40114 78540 40012 78590 40117 78595 78740 78728 24041 40014
 24084 78750 24069 40115 24019 24009 24046 28040 28080 40011 24049 24042
 24800 28015 29080 29065 78400 29070 26050 40013 24040 24081 26060 28067
 28064 24025 26080 78137 24060 29075 28065 24006 26040 78138 28037 24030
 24062 78769 78535 78570 24045 24043 24070 28063 78575 29050 78135 29074
 26090 28060 12881 78722]

Unique values in column 'fund_description':
['GENERAL FUND' 'SENIOR CITIZENS PROG TF' 'EEOC TRUST FUND' 'GRANTS FUND'
 'POLICE SECONDARY EMPL T.F.' 'MELLON PARK TENNIS TF' 'ARAD-PUBLIC WORKS'
 'HUD-FAIR HOUSING PROG TF' 'SPEC SUM FOOD SERVICE TF' 'ARAD-PARKS' 'J

## Renaming of data for consistency

In [51]:
replace_dict_fund_description = {
    'GENERAL FUND': 'General Fund',
    'SENIOR CITIZENS PROG TF': 'Senior Citizens Program Trust Fund',
    'EEOC TRUST FUND': 'Equal Employment Opportunity Commission Trust Fund',
    'GRANTS FUND': 'Grants Fund',
    'POLICE SECONDARY EMPL T.F.': 'Police Secondary Employment Trust Fund',
    'MELLON PARK TENNIS TF': 'Mellon Park Tennis Trust Fund',
    'ARAD-PUBLIC WORKS': 'Allegheny Regional Asset District Public Works',
    'HUD-FAIR HOUSING PROG TF': 'Housing and Urban Development Fair Housing Program Trust Fund',
    'SPEC SUM FOOD SERVICE TF': 'Special Summer Food Service Trust Fund',
    'ARAD-PARKS': 'Allegheny Regional Asset District Parks',
    'JTPA': 'Job Training Partnership Act',
    'TREE TAXING BODIES': 'Tree Taxing Bodies',
    'CDBG FUND': 'Community Development Block Grant Fund',
    'EMS REIMB EVENTS TRUST FUND': 'Emergency Medical Services Reimbursement Events Trust Fund',
    '3 TAXING BODIES ESCROW TF': 'Three Taxing Bodies Escrow Trust Fund',
    'DEBT SERVICE- PROJ FUND': 'Debt Service Project Fund',
    'WORKERS\' COMP FUND': 'Workers Compensation Fund',
    'CONFISCATED NARC PROCEEDS': 'Confiscated Narcotics Proceeds',
    'EMERGENCY SHELTERS PROG TF': 'Emergency Shelters Program Trust Fund',
    'SOLID WASTE TF': 'Solid Waste Trust Fund',
    'PUBLIC WORKS TRUST FUND': 'Public Works Trust Fund',
    'SHADE TREE TRUST FUND': 'Shade Tree Trust Fund',
    '2020 PAYGO': '2020 Pay-As-You-Go',
    'SPECIAL PKS PROG/GREAT RACE': 'Special Parks Program Great Race',
    'WORKERS\' COMP COMMUTATIONS': 'Workers Compensation Commutations',
    'VEBA- HEALTH INSURANCE': 'Voluntary Employees Beneficiary Association Health Insurance',
    'VEBA-WORKERS\' COMP': 'Voluntary Employees Beneficiary Association Workers Compensation',
    'PROJECT FUND 2017 BOND': 'Project Fund 2017 Bond',
    'PROJECT FUND 2018 BOND': 'Project Fund 2018 Bond',
    '2019 PAYGO': '2019 Pay-As-You-Go',
    'PROJECT FUND 2019 BOND': 'Project Fund 2019 Bond',
    '2018 PAYGO': '2018 Pay-As-You-Go',
    'PEMA AND HOMELAND SECURITY': 'Pennsylvania Emergency Management Agency and Homeland Security',
    'GREEN INITIATIVE TRUST FUND': 'Green Initiative Trust Fund',
    'CAP PROJ - PROJ FUND': 'Capital Project Fund',
    'SCHENLEY PARK RINK TF': 'Schenley Park Rink Trust Fund',
    'LIQUID FUELS TAX': 'Liquid Fuels Tax',
    '2016 PAYGO': '2016 Pay-As-You-Go',
    'PROJECT FUND 2020 BOND': 'Project Fund 2020 Bond',
    'TIP PROJECTS': 'Transportation Improvement Program Projects',
    'FIRE ESCROW': 'Fire Escrow',
    'PROJECT FUND 2015 BOND': 'Project Fund 2015 Bond',
    'Project Fund-2012 Bond': 'Project Fund 2012 Bond',
    'REAL ESTATE TAX REFUNDS - CITY': 'Real Estate Tax Refunds City',
    '2017 PAYGO': '2017 Pay-As-You-Go',
    'REAL ESTATE TAX REFUNDS-SCHOOL': 'Real Estate Tax Refunds School',
    'EMPL TRAVEL EXPENSE ADVANCE TF': 'Employee Travel Expense Advance Trust Fund',
    'DEMOLITION TF': 'Demolition Trust Fund',
    'HAZARD MAERIALS TRUST FUND': 'Hazardous Materials Trust Fund',
    '2014 PAYGO': '2014 Pay-As-You-Go',
    'CROSSING GUARD SPECIAL EVENTS': 'Crossing Guard Special Events',
    'PITTSBURGH CODE TF': 'Pittsburgh Code Trust Fund',
    'PUBLIC SAFETY TRAINING TF': 'Public Safety Training Trust Fund',
    '2015 PAYGO': '2015 Pay-As-You-Go',
    'CONFISCATED NON-NARC PROCEEDS': 'Confiscated Non-Narcotics Proceeds',
    'CODE TRUST FUND': 'Code Trust Fund',
    'MICROFILM PERMIT PLANS TF': 'Microfilm Permit Plans Trust Fund',
    'PGH COMM ON HUMAN TELA TF': 'Pittsburgh Commission on Human Relations Trust Fund',
    'VERIZON FRANCHISE TF': 'Verizon Franchise Trust Fund',
    'CAPITAL PROJECTS - 2006 BOND': 'Capital Projects 2006 Bond',
    'Mounted Police TF': 'Mounted Police Trust Fund',
    'GRAFFITI TRUST FUND': 'Graffiti Trust Fund',
    'SOUTH SIDE PARKING ENHANCEMENT': 'South Side Parking Enhancement',
    'COMCAST FRANCHISE TF': 'Comcast Franchise Trust Fund',
    'FRICK PARK TRUST FUND': 'Frick Park Trust Fund',
    'PUBLIC SAFETY SUPPORT TRUST FU': 'Public Safety Support Trust Fund',
    'ESCHEAT FUND': 'Escheat Fund',
    'DISTRICT 9 CRIME PREVENTION': 'District 9 Crime Prevention',
    'HOPWA': 'Housing Opportunities for Persons With AIDS',
    '2013 PAYGO': '2013 Pay-As-You-Go',
    'FEDERAL TASK FORCE TF': 'Federal Task Force Trust Fund',
    'YCPC/MAYOR\'S YOUTH INIT TF': 'York County Planning Commission Mayor\'s Youth Initiative Trust Fund',
    'ARC': 'Appalachian Regional Commission',
    'WORKFORCE DEVELOPMENT FUND': 'Workforce Development Fund',
    'STORMWATER MANAGEMENT TRUST FU': 'Stormwater Management Trust Fund',
    'DURG ABUSE RESISTANCE ED TF': 'Drug Abuse Resistance Education Trust Fund',
    'Brownfield Economic Developmen': 'Brownfield Economic Development Fund',
    'OPEB TRUST-POLICE': 'Other Post-Employment Benefits Trust Fund Police',
    'ARCHIVES AND RECORDS MANAGEMEN': 'Archives and Records Management',
    'OPERATION WEED & SEED FUND': 'Operation Weed and Seed Fund',
    'Facilities Trust Fund': 'Facilities Trust Fund',
    'WAYFINDERS SIGNAGE TF': 'Wayfinders Signage Trust Fund',
    'CRIMINAL INTELLIGENCE': 'Criminal Intelligence Fund',
    'STATE EMERGENCY SHELTER PROG': 'State Emergency Shelter Program',
    'OPEB TRUST-FIRE': 'Other Post-Employment Benefits Trust Fund Fire',
    'PHIPPS CONSERVATORY': 'Phipps Conservatory Fund',
    'AUTO THEFT TF II': 'Auto Theft Task Force II Trust Fund',
    'PANGIS TRUST FUND': 'Pittsburgh Area National Geospatial-Intelligence Agency Trust Fund',
    'VENDING TF-CONTROLLER': 'Vending Trust Fund Controller',
    'ETHICS BOARD TRUST FUND': 'Ethics Board Trust Fund',
    'EARNED INCOME TAX REFUNDS-CITY': 'Earned Income Tax Refunds City',
    'LOCAL LAW ENFORCEMENT': 'Local Law Enforcement Fund',
    'Project Fund 2015 Bond Issue': 'Project Fund 2015 Bond Issue',
    'HIGHWAY SAFETY TF': 'Highway Safety Trust Fund',
    'RENTAL PERMIT TF': 'Rental Permit Trust Fund',
    'OPEN SPACE TRUST FUND': 'Open Space Trust Fund',
    'EARNED INC TAX REFUNDS-SCHOOL': 'Earned Income Tax Refunds School',
    'STIMULUS FUND': 'Stimulus Fund',
    'VENDING TF-COUNCIL': 'Vending Trust Fund Council',
    'OPEB TRUST FUND': 'Other Post-Employment Benefits Trust Fund',
    'LIBERTY AVE BIKE LANE GRANT': 'Liberty Avenue Bike Lane Grant',
    'HPRP': 'Homelessness Prevention and Rapid Re-Housing Program',
    'SWIMMING POOL MAINT TF': 'Swimming Pool Maintenance Trust Fund',
    'COPIER MANAGEMENT & POSTAGE TF': 'Copier Management and Postage Trust Fund',
    'CONTROLLER\'S FIN MAGMT. I.S.': 'Controller\'s Financial Management Information System'
}

df['fund_description'] = df['fund_description'].replace(replace_dict_fund_description)

replace_dict_department_name = {
    'Innovation & Performance': 'Innovation and Performance',
    'DPS-Police': 'Department of Public Safety - Police',
    'DPS-Fire': 'Department of Public Safety - Fire',
    'DPW-Operations': 'Department of Public Works - Operations',
    'Permits Licenses and Inspectio': 'Permits Licenses and Inspection',
    'DPS-Administration': 'Department of Public Safety - Administration',
    'OMI': 'Office of Municipal Investigations',
    'DPS-Animal Care and Control': 'Department of Public Safety - Animal Care and Control',
    'DPW-Administration': 'Department of Public Works - Administration',
    'DPW- Environmental Services': 'Department of Public Works - Environmental Services',
    'DPW - Facilities': 'Department of Public Works - Facilities',
    'Office of  Management and Budg': 'Office of Management and Budget',
    'DPS-Emergency Medical Services': 'Department of Public Safety - Emergency Medical Services',
    'URA Projects': 'Urban Redevelopment Authority Projects',
    'Equipment Leasing Authority': 'Equipment Leasing Authority',
    'Equal Opportunity Review Commi': 'Equal Opportunity Review Committee',
    'Finance, Procurement, Fleet, a': 'Finance, Procurement and Fleet',
    'Non Departmental': 'Non Departmental',
    'DPW-Transportation and Engine': 'Department of Public Works - Transportation and Engine',
    'DPS-Building Inspection': 'Department of Public Safety - Building Inspection',
}

df['department_name'] = df['department_name'].replace(replace_dict_department_name)

# Get the first description found for each cost_center_id
mapping = df.groupby('cost_center_id')['cost_center_description'].first().to_dict()

# Replace all descriptions with the first one found for each ID
df['cost_center_description'] = df['cost_center_id'].map(mapping)


In [52]:
# Get unique values for each column
for column in df.columns:
    unique_values = df[column].unique()
    print(f"\nUnique values in column '{column}':")
    print(unique_values)


Unique values in column 'id':
[77837069 77837070 77837071 ... 78454581 78454582 78454583]

Unique values in column 'fund_id':
[11101 28050 28020 29040 24067 28033 28003 26030 28055 28004 23000 78765
 26010 24037 78768 32000 24016 26020 28070 28043 28053 40120 29060 75020
 78160 78170 40017 40018 40119 40019 40118 24063 28085 40001 28045 22000
 40116 40020 40114 78540 40012 78590 40117 78595 78740 78728 24041 40014
 24084 78750 24069 40115 24019 24009 24046 28040 28080 40011 24049 24042
 24800 28015 29080 29065 78400 29070 26050 40013 24040 24081 26060 28067
 28064 24025 26080 78137 24060 29075 28065 24006 26040 78138 28037 24030
 24062 78769 78535 78570 24045 24043 24070 28063 78575 29050 78135 29074
 26090 28060 12881 78722]

Unique values in column 'fund_description':
['General Fund' 'Senior Citizens Program Trust Fund'
 'Equal Employment Opportunity Commission Trust Fund' 'Grants Fund'
 'Police Secondary Employment Trust Fund' 'Mellon Park Tennis Trust Fund'
 'Allegheny Regional As

## Check first and last date of entry recorded in dataset

In [53]:
# Convert 'general_ledger_date' to datetime if it's not already
df['general_ledger_date'] = pd.to_datetime(df['general_ledger_date'])

# Find the first date (minimum date)
first_date = df['general_ledger_date'].min()

# Find the last date (maximum date)
last_date = df['general_ledger_date'].max()

# Print the results
print(f"First date: {first_date}")
print(f"Last date: {last_date}")

First date: 2012-01-03 00:00:00
Last date: 2020-12-31 00:00:00


C:\Users\User\AppData\Local\Temp\ipykernel_24000\1323043655.py:2: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df['general_ledger_date'] = pd.to_datetime(df['general_ledger_date'])


In [54]:
# Function to standardize the 'fund_description' column
def standardize_fund_description(fund_description):
    return (fund_description
            .replace('-', '_')
            .replace(' ', '_')
            .replace('.', '')
            .replace('\'', '')
            .replace('/', '')
            .replace('&', 'AND')
            .upper())

# Apply the function to the 'fund_description' column
df['fund_description_key'] = df['fund_description'].apply(standardize_fund_description)

# Dictionary to map old values to new ones
replace_dict_fund_description_key = {
    'DEBT_SERVICE__PROJ_FUND': 'DEBT_SERVICE_PROJ_FUND',
    'CAP_PROJ___PROJ_FUND': 'CAP_PROJ_PROJ_FUND',
    'REAL_ESTATE_TAX_REFUNDS___CITY': 'REAL_ESTATE_TAX_REFUNDS_CITY',
    'CAPITAL_PROJECTS___2006_BOND': 'CAPITAL_PROJECTS_2006_BOND'
}

# Apply the replacements
df['fund_description_key'] = df['fund_description_key'].replace(replace_dict_fund_description_key)

column = 'fund_description_key'
unique_values = df[column].unique()
print(f"\nUnique values in column '{column}':")
for i in unique_values:
    print(i)


Unique values in column 'fund_description_key':
GENERAL_FUND
SENIOR_CITIZENS_PROGRAM_TRUST_FUND
EQUAL_EMPLOYMENT_OPPORTUNITY_COMMISSION_TRUST_FUND
GRANTS_FUND
POLICE_SECONDARY_EMPLOYMENT_TRUST_FUND
MELLON_PARK_TENNIS_TRUST_FUND
ALLEGHENY_REGIONAL_ASSET_DISTRICT_PUBLIC_WORKS
HOUSING_AND_URBAN_DEVELOPMENT_FAIR_HOUSING_PROGRAM_TRUST_FUND
SPECIAL_SUMMER_FOOD_SERVICE_TRUST_FUND
ALLEGHENY_REGIONAL_ASSET_DISTRICT_PARKS
JOB_TRAINING_PARTNERSHIP_ACT
TREE_TAXING_BODIES
COMMUNITY_DEVELOPMENT_BLOCK_GRANT_FUND
EMERGENCY_MEDICAL_SERVICES_REIMBURSEMENT_EVENTS_TRUST_FUND
THREE_TAXING_BODIES_ESCROW_TRUST_FUND
DEBT_SERVICE_PROJECT_FUND
CONFISCATED_NARCOTICS_PROCEEDS
EMERGENCY_SHELTERS_PROGRAM_TRUST_FUND
SOLID_WASTE_TRUST_FUND
PUBLIC_WORKS_TRUST_FUND
SHADE_TREE_TRUST_FUND
2020_PAY_AS_YOU_GO
SPECIAL_PARKS_PROGRAM_GREAT_RACE
WORKERS_COMPENSATION_COMMUTATIONS
VOLUNTARY_EMPLOYEES_BENEFICIARY_ASSOCIATION_HEALTH_INSURANCE
VOLUNTARY_EMPLOYEES_BENEFICIARY_ASSOCIATION_WORKERS_COMPENSATION
PROJECT_FUND_2017_BOND


In [55]:
def standardize_department_name(department_name):
    return (department_name
            .replace('-', '_')
            .replace(' ', '_')
            .replace('.', '')
            .replace('\'', '')
            .replace('/', '_AND_')
            .replace('&', 'AND')
            .upper())

df['department_name_key'] = df['department_name'].apply(standardize_department_name)

replace_dict_department_name_key = {
    'DEPARTMENT_OF_PUBLIC_SAFETY___POLICE': 'DEPARTMENT_OF_PUBLIC_SAFETY_POLICE',
    'DEPARTMENT_OF_PUBLIC_SAFETY___FIRE': 'DEPARTMENT_OF_PUBLIC_SAFETY_FIRE',
    'DEPARTMENT_OF_PUBLIC_WORKS___OPERATIONS': 'DEPARTMENT_OF_PUBLIC_WORKS_OPERATIONS',
    'DEPARTMENT_OF_PUBLIC_SAFETY___ADMINISTRATION': 'DEPARTMENT_OF_PUBLIC_SAFETY_ADMINISTRATION',
    'DEPARTMENT_OF_PUBLIC_SAFETY___ANIMAL_CARE_AND_CONTROL': 'DEPARTMENT_OF_PUBLIC_SAFETY_ANIMAL_CARE_AND_CONTROL',
    'DEPARTMENT_OF_PUBLIC_WORKS___ADMINISTRATION': 'DEPARTMENT_OF_PUBLIC_WORKS_ADMINISTRATION',
    'DEPARTMENT_OF_PUBLIC_WORKS___ENVIRONMENTAL_SERVICES': 'DEPARTMENT_OF_PUBLIC_WORKS_ENVIRONMENTAL_SERVICES',
    'DEPARTMENT_OF_PUBLIC_WORKS___FACILITIES': 'DEPARTMENT_OF_PUBLIC_WORKS_FACILITIES',
    'DEPARTMENT_OF_PUBLIC_SAFETY___EMERGENCY_MEDICAL_SERVICES': 'DEPARTMENT_OF_PUBLIC_SAFETY_EMERGENCY_MEDICAL_SERVICES',
    'DEPARTMENT_OF_PUBLIC_WORKS___TRANSPORTATION_AND_ENGINE': 'DEPARTMENT_OF_PUBLIC_WORKS_TRANSPORTATION_AND_ENGINE',
    'DEPARTMENT_OF_PUBLIC_SAFETY___BUILDING_INSPECTION': 'DEPARTMENT_OF_PUBLIC_SAFETY_BUILDING_INSPECTION',
    'FINANCE,_PROCUREMENT_AND_FLEET': 'FINANCE_PROCUREMENT_AND_FLEET'
}

df['department_name_key'] = df['department_name_key'].replace(replace_dict_department_name_key)

column = 'department_name_key'
unique_values = df[column].unique()
print(f"\nUnique values in column '{column}':")
for i in unique_values:
    print(i)


Unique values in column 'department_name_key':
INNOVATION_AND_PERFORMANCE
DEPARTMENT_OF_FINANCE
DEPARTMENT_OF_PUBLIC_SAFETY_POLICE
DEPARTMENT_OF_PUBLIC_SAFETY_FIRE
DEPARTMENT_OF_PUBLIC_WORKS_OPERATIONS
PARKS_AND_RECREATION
CITY_COUNCIL
CITY_CLERK
MAYORS_OFFICE
HUMAN_RELATIONS_COMMISSION
CONTROLLERS_OFFICE
DEPARTMENT_OF_LAW
PERSONNEL_AND_CIVIL_SERVICE
DEPARTMENT_OF_CITY_PLANNING
PERMITS_LICENSES_AND_INSPECTION
DEPARTMENT_OF_PUBLIC_SAFETY_ADMINISTRATION
OFFICE_OF_MUNICIPAL_INVESTIGATIONS
DEPARTMENT_OF_PUBLIC_SAFETY_ANIMAL_CARE_AND_CONTROL
DEPARTMENT_OF_PUBLIC_WORKS_ADMINISTRATION
DEPARTMENT_OF_PUBLIC_WORKS_ENVIRONMENTAL_SERVICES
DEPARTMENT_OF_PUBLIC_WORKS_FACILITIES
MOBILITY_AND_INFRASTRUCTURE
PUBLIC_WORKS
OFFICE_OF_MANAGEMENT_AND_BUDGET
DEPARTMENT_OF_PUBLIC_SAFETY_EMERGENCY_MEDICAL_SERVICES
URBAN_REDEVELOPMENT_AUTHORITY_PROJECTS
EQUIPMENT_LEASING_AUTHORITY
EQUAL_OPPORTUNITY_REVIEW_COMMITTEE
FINANCE_PROCUREMENT_AND_FLEET
DEPARTMENT_OF_PUBLIC_WORKS_TRANSPORTATION_AND_ENGINE
DEPARTMENT_OF

In [56]:
# Function to standardize the 'cost_center_description' column
def standardize_cost_center_description(cost_center_description):
    return (cost_center_description
            .replace('-', '_')
            .replace(' ', '_')
            .replace('.', '')
            .replace('\'', '')
            .replace('/', '')
            .replace('&', 'AND')
            .replace('__', '_')
            .replace('___', '_')
            .upper())

# Apply the function to the 'cost_center_description' column
df['cost_center_description_key'] = df['cost_center_description'].apply(standardize_cost_center_description)

# # Dictionary to map old values to new ones
# replace_dict_fund_description_key = {
#     'DEBT_SERVICE__PROJ_FUND': 'DEBT_SERVICE_PROJ_FUND',
#     'CAP_PROJ___PROJ_FUND': 'CAP_PROJ_PROJ_FUND',
#     'REAL_ESTATE_TAX_REFUNDS___CITY': 'REAL_ESTATE_TAX_REFUNDS_CITY',
#     'CAPITAL_PROJECTS___2006_BOND': 'CAPITAL_PROJECTS_2006_BOND'
# }

# Apply the replacements
# df['fund_description_key'] = df['fund_description_key'].replace(replace_dict_fund_description_key)

column = 'cost_center_description_key'
unique_values = df[column].unique()
print(f"\nUnique values in column '{column}':")
for i in unique_values:
    print(i)


Unique values in column 'cost_center_description_key':
INNOVATION_AND_PERFORMANCE
FINANCE
PS__POLICE_BUREAU
PS__FIRE_BUREAU
PW_BUREAU_OF_PW_OPERATIONS
PARKS_AND_RECREATION
CITY_COUNCIL
CITY_CLERK
MAYORS_OFFICE
BUREAU_OF_NEIGHBORHOOD_EMPOWER
OFFICE_OF_MANAGEMENT_AND_BUDGE
HUMAN_RELATIONS_COMM
CITY_CONTROLLER
LAW
ETHICS_BOARD
HUMAN_RESOURCESCIVIL_SERVICE
CITY_PLANNING
PERMITS_LICENSES_AND_INSPECTIO
PS__ADMIN_AND_SUPPORT_SERVICE
OFFICE_OF_MUNICIPAL_INVESTIGAT
PS__BUREAU_OF_ANIMAL_CARE_AND
PW_BUREAU_OF_ADMINISTRATION
PW_ENVIRONMENTAL_SERVICES
BUREAU_OF_FACILITIES
MOBILITY_AND_INFRASTRUCTURE
SENIOR_CITEZENS_PROG_TF
EEOC_TRUST_FUND
DOMI_HEINZ_GRANT
POLICE_SECONDARY_EMPL_TF
DOMESTIC_VIOLENCE_UNIT
HILLMAN_TRANSPORTATION_FELLOW
KNIGHT_FOUNDATION_AUTONOMOUS_V
MELLON_PARK_TENNIS_TF
ARAD_PUBLIC_WORKS
BRT_POSITION
HUD_FAIR_HOUSING_PROG_TF
SPEC_SUM_FOOD_SERVICE_TF
ARAD_PARKS
JTPA
THREE_TAXING_BODIES_FUND
CDBG_PERSONNEL
PS__EMERGENCY_MED_SERVICES
EMS_REIMB_EVENTS_TRUST_FUND
3_TAXING_BODIES_ESCROW_TF

In [58]:
# Function to standardize the 'cost_center_description' column
def standardize_cost_center_description(cost_center_description):
    return (cost_center_description
            .replace('__', '_')
            .replace('___', '_')
            .upper())

# Apply the function to the 'cost_center_description' column
df['cost_center_description_key'] = df['cost_center_description_key'].apply(standardize_cost_center_description)

column = 'cost_center_description_key'
unique_values = df[column].unique()
print(f"\nUnique values in column '{column}':")
for i in unique_values:
    print(i)


Unique values in column 'cost_center_description_key':
INNOVATION_AND_PERFORMANCE
FINANCE
PS_POLICE_BUREAU
PS_FIRE_BUREAU
PW_BUREAU_OF_PW_OPERATIONS
PARKS_AND_RECREATION
CITY_COUNCIL
CITY_CLERK
MAYORS_OFFICE
BUREAU_OF_NEIGHBORHOOD_EMPOWER
OFFICE_OF_MANAGEMENT_AND_BUDGE
HUMAN_RELATIONS_COMM
CITY_CONTROLLER
LAW
ETHICS_BOARD
HUMAN_RESOURCESCIVIL_SERVICE
CITY_PLANNING
PERMITS_LICENSES_AND_INSPECTIO
PS_ADMIN_AND_SUPPORT_SERVICE
OFFICE_OF_MUNICIPAL_INVESTIGAT
PS_BUREAU_OF_ANIMAL_CARE_AND
PW_BUREAU_OF_ADMINISTRATION
PW_ENVIRONMENTAL_SERVICES
BUREAU_OF_FACILITIES
MOBILITY_AND_INFRASTRUCTURE
SENIOR_CITEZENS_PROG_TF
EEOC_TRUST_FUND
DOMI_HEINZ_GRANT
POLICE_SECONDARY_EMPL_TF
DOMESTIC_VIOLENCE_UNIT
HILLMAN_TRANSPORTATION_FELLOW
KNIGHT_FOUNDATION_AUTONOMOUS_V
MELLON_PARK_TENNIS_TF
ARAD_PUBLIC_WORKS
BRT_POSITION
HUD_FAIR_HOUSING_PROG_TF
SPEC_SUM_FOOD_SERVICE_TF
ARAD_PARKS
JTPA
THREE_TAXING_BODIES_FUND
CDBG_PERSONNEL
PS_EMERGENCY_MED_SERVICES
EMS_REIMB_EVENTS_TRUST_FUND
3_TAXING_BODIES_ESCROW_TF
DEBT

In [59]:
column = 'department_name'
unique_values = df[column].unique()
print(f"\nUnique values in column '{column}':")
for i in unique_values:
    print(i)


Unique values in column 'department_name':
Innovation and Performance
Department of Finance
Department of Public Safety - Police
Department of Public Safety - Fire
Department of Public Works - Operations
Parks and Recreation
City Council
City Clerk
Mayor's Office
Human Relations Commission
Controller's Office
Department of Law
Personnel/Civil Service
Department of City Planning
Permits Licenses and Inspection
Department of Public Safety - Administration
Office of Municipal Investigations
Department of Public Safety - Animal Care and Control
Department of Public Works - Administration
Department of Public Works - Environmental Services
Department of Public Works - Facilities
Mobility and Infrastructure
Public Works
Office of Management and Budget
Department of Public Safety - Emergency Medical Services
Urban Redevelopment Authority Projects
Equipment Leasing Authority
Equal Opportunity Review Committee
Finance, Procurement and Fleet
Department of Public Works - Transportation and Engin

In [60]:
column = 'fund_description'
unique_values = df[column].unique()
print(f"\nUnique values in column '{column}':")
for i in unique_values:
    print(i)


Unique values in column 'fund_description':
General Fund
Senior Citizens Program Trust Fund
Equal Employment Opportunity Commission Trust Fund
Grants Fund
Police Secondary Employment Trust Fund
Mellon Park Tennis Trust Fund
Allegheny Regional Asset District Public Works
Housing and Urban Development Fair Housing Program Trust Fund
Special Summer Food Service Trust Fund
Allegheny Regional Asset District Parks
Job Training Partnership Act
Tree Taxing Bodies
Community Development Block Grant Fund
Emergency Medical Services Reimbursement Events Trust Fund
Three Taxing Bodies Escrow Trust Fund
Debt Service Project Fund
Confiscated Narcotics Proceeds
Emergency Shelters Program Trust Fund
Solid Waste Trust Fund
Public Works Trust Fund
Shade Tree Trust Fund
2020 Pay-As-You-Go
Special Parks Program Great Race
Workers Compensation Commutations
Voluntary Employees Beneficiary Association Health Insurance
Voluntary Employees Beneficiary Association Workers Compensation
Project Fund 2017 Bond
Proj

## Extracting out necessary information only

In [66]:
# Getting only the necessary columns for fund_df
fund_df = df[['fund_id', 'fund_description', 'fund_description_key']].drop_duplicates()
department_df = df[['department_id', 'department_name', 'department_name_key']].drop_duplicates()
cost_center_df = df[['cost_center_id', 'cost_center_description', 'cost_center_description_key']].drop_duplicates()
ledger_df = df[['id', 'general_ledger_date', 'amount', 'ledger_description', 'fund_id', 'department_id','cost_center_id']]

## Final check if dataset is clean

In [67]:
# Check if ledger_df.id is unique
is_ledger_id_unique = ledger_df['id'].nunique() == len(ledger_df)
print(f"Are all ledger_df.id values unique? {is_ledger_id_unique}")
print(f"Ledger df shape? {ledger_df.shape}")

# Check if fund_df.fund_id is unique
is_fund_id_unique = fund_df['fund_id'].nunique() == len(fund_df)
print(f"Are all fund_df.fund_id values unique? {is_fund_id_unique}")

# Check if department_df.department_id is unique
is_department_id_unique = department_df['department_id'].nunique() == len(department_df)
print(f"Are all department_df.department_id values unique? {is_department_id_unique}")


# Check if cost_center_df.cost_center_id is unique
is_cost_center_id_unique = cost_center_df['cost_center_id'].nunique() == len(cost_center_df)
print(f"Are all cost_center_df.cost_center_id values unique? {is_cost_center_id_unique}")

Are all ledger_df.id values unique? True
Ledger df shape? (609499, 7)
Are all fund_df.fund_id values unique? True
Are all department_df.department_id values unique? True
Are all cost_center_df.cost_center_id values unique? True


## If dataset has duplicate values in the id column, find those entries 

In [68]:
# Identify non-unique fund_ids in fund_df
non_unique_fund_ids = fund_df[fund_df['fund_id'].duplicated(keep=False)]

# Display the non-unique rows
print(non_unique_fund_ids)

# Identify non-unique department_ids in department_df
non_unique_department_ids = department_df[department_df['department_id'].duplicated(keep=False)]

# Display the non-unique rows
print(non_unique_department_ids)

# Identify non-unique department_ids in department_df
non_unique_cost_center_ids = cost_center_df[cost_center_df['cost_center_id'].duplicated(keep=False)]

# Display the non-unique rows
print(non_unique_cost_center_ids)

Empty DataFrame
Columns: [fund_id, fund_description, fund_description_key]
Index: []
Empty DataFrame
Columns: [department_id, department_name, department_name_key]
Index: []
Empty DataFrame
Columns: [cost_center_id, cost_center_description, cost_center_description_key]
Index: []


## Save the dataset to their respective path in the current directory

In [69]:
ledger_df.to_csv('dataset/ledger.csv', index=False)
fund_df.to_csv('dataset/fund.csv', index=False)
department_df.to_csv('dataset/department.csv', index=False)
cost_center_df.to_csv('dataset/cost_center.csv', index=False)

In [ ]:
import pandas as pd

# Load your CSV files
department_df = pd.read_csv('dataset/department.csv')
fund_df = pd.read_csv('dataset/fund.csv')
ledger_df = pd.read_csv('dataset/ledger.csv')
cost_center_df = pd.read_csv('dataset/cost_center.csv')

# Function to convert DataFrame to SQL INSERT statements and split into files
def df_to_sql_insert_split(df, table_name, max_rows_per_file=1000000):
    insert_statements = []
    file_count = 1
    for index, row in df.iterrows():
        columns = ', '.join(row.index)
        values = ', '.join([f"'{str(value)}'" if isinstance(value, str) else str(value) for value in row.values])
        insert_statements.append(f"INSERT INTO {table_name} ({columns}) VALUES ({values});")
        
        # If we have reached the max rows per file, write to a new file
        if (index + 1) % max_rows_per_file == 0 or index == len(df) - 1:
            filename = f'insert_{table_name}_sql_statements_part_{file_count}.sql'
            with open(filename, 'w') as file:
                file.write('USE revex_insights;\n' + '\n'.join(insert_statements))
            insert_statements = []  # Reset for the next file
            file_count += 1

# Convert DataFrames to SQL INSERT statements and split into files
df_to_sql_insert_split(department_df, 'department')
df_to_sql_insert_split(fund_df, 'fund')
df_to_sql_insert_split(ledger_df, 'ledger')
df_to_sql_insert_split(cost_center_df, 'cost_center')

In [ ]:
import pandas as pd

# Load your CSV files
department_df = pd.read_csv('dataset/department.csv')
fund_df = pd.read_csv('dataset/fund.csv')
ledger_df = pd.read_csv('dataset/ledger.csv')
cost_center_df = pd.read_csv('dataset/cost_center.csv')

# Function to convert DataFrame to SQL INSERT statements and split into files
def df_to_sql_insert_split(df, table_name, max_rows_per_file=1000000):
    insert_statements = []
    file_count = 1
    
    for index, row in df.iterrows():
        columns = ', '.join(row.index)  # Column names
        
        values = []
        for value in row.values:
            if isinstance(value, str):
                escaped_value = value.replace("'", "''")  # Escape single quotes
                values.append(f"'{escaped_value}'")
            else:
                values.append(str(value))  # Keep numbers as they are
        
        values_str = ', '.join(values)
        insert_statements.append(f"INSERT INTO {table_name} ({columns}) VALUES ({values_str});")
        
        # If we have reached max rows per file, write to a new file
        if (index + 1) % max_rows_per_file == 0 or index == len(df) - 1:
            filename = f'insert_{table_name}_sql_statements_part_{file_count}.sql'
            with open(filename, 'w') as file:
                file.write('USE revex_insights;\n' + '\n'.join(insert_statements) + '\n')
            insert_statements = []  # Reset for the next file
            file_count += 1

# Convert DataFrames to SQL INSERT statements and split into files
df_to_sql_insert_split(department_df, 'department')
df_to_sql_insert_split(fund_df, 'fund')
df_to_sql_insert_split(ledger_df, 'ledger')
df_to_sql_insert_split(cost_center_df, 'cost_center')
